# Pipeline 1

In [1]:
from pipeline import Falcon2Linker, SerialAnnotator, T5Converter
import time
from pprint import pprint

[Falcon2.0 main.py]: no reranking, k=1


In [2]:
from pathlib import Path
import pandas as pd
import json
import os

data_path = Path("..") / "t5-for-sparql" / "data" / "lcquad2" / "train.json"
data2_path = Path("..") / "t5-for-sparql" / "data" / "lcquad2" / "test_KG_4211.json"
output_links_dir = Path("..") / "t5-for-sparql" / "falcon_links" / "1ents_1rels"
os.environ['ES_ENTITY_INDEX'] = "wikidataentityindex"
os.environ['ES_PROPERTY_INDEX'] = "wikidatapropertyindex"

In [3]:
linker = Falcon2Linker()
annotator = SerialAnnotator()
converter = T5Converter()

In [4]:
utterance = "Who is the wife of Barack Obama"

In [5]:
linked = linker.link(utterance)
# rules = [1,2,3,4,5,8,9,10,12,13,14]
# linked = process_text_E_R(utterance, rules)
pprint("Linked")
pprint(linked)

pprint("Annotated")
annotated = annotator.annotate(**linked)
pprint(annotated)

pprint("Converted")
converted = converter.preprocess_inputs(**annotated)
pprint(converted)

recieved utterance
0 : ['Who is the wife of Barack Obama']


['Who is the wife of Barack Obama', [['<http://www.wikidata.org/entity/P26>', 'wife']], [['<http://www.wikidata.org/entity/Q76>', 'Barack obama']], 0, 0, 0, 0]
'Linked'
{'ents': [{'id': 'Q76',
           'prefix': 'wd:',
           'uri': 'http://www.wikidata.org/entity/Q76'}],
 'rels': [{'id': 'P26',
           'prefix': 'wdt:',
           'uri': 'http://www.wikidata.org/prop/direct/P26'}],
 'utterance': 'Who is the wife of Barack Obama'}
'Annotated'


{'fragments': ['[DEF]', 'wd:', 'Q76 Barack', '[DEF]', 'wdt:', 'P26 partner'],
 'utterance': 'Who is the wife of Barack Obama'}
'Converted'
('Who is the wife of Barack Obama <extra_id_59> <extra_id_53> Q76 Barack '
 '<extra_id_59> <extra_id_54> P26 partner')


In [6]:
import re

def retrieve_gold_links(masked_wikisparql):
  fragments = masked_wikisparql.upper().strip().split(' ')
  ents = []
  rels = []
  result_dict = {
    'ents': ents,
    'rels': rels,
  }
  for fragment in fragments:
    id_match = re.search('[QP][0-9]+',fragment)
    if id_match:
      id_raw = id_match.group(0)
      if id_raw[0] == 'Q':
        prefix = 'wdt:'
        uri = 'http://www.wikidata.org/entity/' + id_raw
        target_arr = ents
      elif id_raw[0] == 'P':
        prefix = 'wd:'
        uri =  'http://www.wikidata.org/prop/direct/' + id_raw
        target_arr = rels
      else:
        raise ValueError('ID does not start with P or Q')
      match_dict = {
        'id' : id_raw,
        'prefix' : prefix,
        'uri' : uri,
      }
      target_arr.append(match_dict)
  return result_dict

In [7]:
def pipe(utterance, wikisparql):
    linked = linker.link(utterance)
    annotated = annotator.annotate(**linked)
    converted = converter.preprocess(**annotated, wikisparql=wikisparql)
    return linked, annotated, converted

In [8]:
def pipe_batch(utterances, wikisparqls):
  batched = []
  link_batch_start = time.time()
  linked = linker.link_batch(utterances)

  assert len(linked) == len(wikisparqls)

  # for i, link in enumerate(linked):
  #   gold_linked = retrieve_gold_links(wikisparqls[i])
  #   new_ents=link['ents']
  #   old_ents_set = set(x['id'] for x in link['ents'])
  #   for gold_ent in gold_linked['ents']:
  #       if gold_ent['id'] not in old_ents_set:
  #           new_ents.append(gold_ent)
  #   link['rels'] = []
  #   assert len(linked[i]['rels']) == 0

  link_batch_end = time.time()
  s = time.time()
  annotated = annotator.batch_annotate(linked)
  e = time.time()
  assert len(linked) == len(annotated)
  annotator_time = e - s
  converter_time = 0
  for i, single_annotated in enumerate(annotated):
    # annotated = annotator.annotate(**single_linked)
    # try:
    #   annotator_time += e - s
    # except Exception as e:
    #   print("[Annotator Error]:", e)
    #   continue
    s = time.time()
    try:
      converted = converter.preprocess(**single_annotated, wikisparql=wikisparqls[i])
    except Exception as e:
      print("[Converter Error]:", e)
      continue
      
    e = time.time()
    converter_time += e - s
    batched.append([linked[i], annotated[i], converted])
  print("Link batch time:", link_batch_end - link_batch_start)
  print("Anno batch time:", annotator_time)
  print("Conv batch time:", converter_time)
  return batched

## Data

In [9]:
with open(data_path) as f:
  data_dict = json.load(f)

with open(data2_path) as f:
  data_dict.extend(json.load(f))

count = 0

for item in data_dict:
  paraphrased_question = item["paraphrased_question"]
  question = item["question"]
  if paraphrased_question == None:
    raise Exception
  if question == None:
    count += 1
    # print(len(paraphrased_question) > 2)
    # pprint(item)
  if item["sparql_wikidata"] == None:
    raise Exception
  

  if len(paraphrased_question) > 2000:
    paraphrased_question = question
  # item["new_question"] = paraphrased_question if len(paraphrased_question) > 2 and paraphrased_question != [] else question
  item["new_question"] = question

print("Missing question count:", count)

df = pd.DataFrame.from_dict(data_dict)
df = df.dropna(axis=0, subset=['new_question'])
df = df[~df.question.isin(['n/a', None, 'none', 'None'])]
n_entries = len(df)
print("Number of rows:", n_entries)

Missing question count: 71
Number of rows: 28247


In [10]:
assert(len(df[df.question.isna()]) == 0)

In [11]:
for data in df[0:5].iterrows():
    print(data[1].new_question)

What periodical literature does Delta Air Lines use as a moutpiece?
Who is the child of Ranavalona I's husband?
Is it true Jeff_Bridges occupation Lane Chandler and photographer ?
What is the pre-requisite of phase matter of Galinstan?
Which is the operating income for Qantas?


In [12]:
responses = []
batch_qns = []
batch_ans = []
truncated_data = df[:]
total_len = len(truncated_data)
last = 0
cutoff = 0

for i, data in enumerate(truncated_data.iterrows()):
    data = data[1]
    if i < cutoff:
        continue
    # print("Pipeline iter", i)
    question = data["new_question"]
    answer = data["sparql_wikidata"]
    # try
    #     linked, annotated, converted = pipe(question, answer)
    # except Exception as e:
    #     print(e)
    #     continue
    # responses.append([linked, annotated, converted])
    if (question and answer) and len(question) >= 2:
        batch_qns.append(question)
        batch_ans.append(answer)
    if ((i + 1) % 2000) == 0 or i == total_len - 1:
        print("\n[Pipeline1]:", f"Linking {last}-{i}")
        try:
            for linked, annotated, converted in pipe_batch(batch_qns, batch_ans):
                responses.append([linked, annotated, converted])
            batch_qns = []
            batch_ans = []
            last = i
        except Exception as e:
            raise e
            # print(e)
            # continue
        with open(output_links_dir / f"link_{i}.json", "w") as f:
            json.dump(responses, f, indent=2, separators=(',',':'))


[Pipeline1]: Linking 0-1999
1 : ['What periodical literature does Delta Air Lines use as a moutpiece?']
1 : ['Which is HanCinema person ID for Zhang Ziyi?']
1 : ['Who is the student that coined the Euler-Lagrange equation?']
1 : ['Does St. Peters Junior School maximum age equal to 11?']
1 : ['Which is the meta-organization that is regulated by the football association?']
1 : ['What award did Olga Tokarczuk receive in 2015?']
1 : ['Who is {champ} of {prize awarded} {Lily Tomlin} ?']
1 : ['Mention the musical related work highlighting the work of Sasha Grey.']
1 : ['What is the Velká pardubická with the lowest race time whose winner is Peter Gehm?']
1 : ['Which is the author of afterword of Erich Fromm?']
1 : ['Is tuberculosis prevalent?']
1 : ['Tell me soil type whose name has the word vertisol in it.']
1 : ['How far west are we?']
1 : ['Who is the mother and husband of Candice Bergen?']
1 : ['Who was presumably thought to be a student of Strabo?']
1 : ['Name a fruit composed of Citrul

1 : ['Is the apoapsis of the Aotearoa 3400 equal to 2.126?']
1 : ['Who regulates the genetic association of hsa-miR-5006-3p in molecular biology']
1 : ['At what role did John Tyler replace William Henry Harrison?']


2 : ['Who is {creators} of {language of the original work} {Star Trek Into Darkness} ?']
3 : ['Tell me national association football team whose name has the word team in it.']


4 : ['What is the active ingredient pharmaceutical product in erythromycin?']


5 : ['Did Judi Densch receive a Tony Award for best direction of the play Premium Imperiale?']


...

28214 : ['What is the depiction of the narrative of Thomas Buddenbrook?']


28216 : ['What musical did Tyne Daly win a Tony award for?']
28217 : ['When was Nice had twinned administrative body of Edinburgh?']
28221 : ['who public office for position held of Moon Jae-in?']


28228 : ['What are the unique signs of a sulfuric acid burn?']
28229 : ['Who is the {missionary society} for {parent organization} of {Catholic Church}']


28237 : ['Is Diabetes mellitus studied in diabetology?']


Link batch time: 50.164427518844604
Anno batch time: 307.1500325202942
Conv batch time: 0.016704082489013672
